# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.duplicated().sum()

0

In [6]:
data['calls'] = data['calls'].astype(int)
data['messages'] = data['messages'].astype(int)

В данных нет пропусков, нет дубликатов, заменили колонки звонков и сообщений на целый тип.

## Разбейте данные на выборки

Разобьем данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [9]:
features1, features_test, target1, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345
)

features_train, features_valid, target_train, target_valid = train_test_split(
    features1, target1, test_size=0.25, random_state=12345
)

print('{:.0%} {:.0%} {:.0%}'.format(len(features_train) / 3214, len(features_valid) / 3214, len(features_test) / 3214))

60% 20% 20%


## Исследуйте модели

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 

In [11]:
best_model = None
best_result = 0
for depth in range(1, 11):
    for split in range(2, 6):
        for leaf in range(1, 6):
            for nodes in range (2, 11):
                model_tree = DecisionTreeClassifier(
                    min_samples_split=split, min_samples_leaf=leaf, max_leaf_nodes=nodes,
                    max_depth=depth, random_state=12345
                )
                model_tree.fit(features_train, target_train)
                result = model_tree.score(features_valid, target_valid)
                if result > best_result:
                    best_model_tree = model_tree
                    best_result = result
                    best_depth = depth
                    best_min_samples_split = split
                    best_min_samples_leaf = leaf
                    best_max_leaf_nodes = nodes
print(best_result, best_min_samples_split, best_min_samples_leaf, best_max_leaf_nodes, best_depth)

0.7776049766718507 2 1 9 7


Accuracy лучшей модели равна 0.7776049766718507 с гиперпараметрами min_samples_split = 2, min_samples_leaf = 1, max_leaf_nodes = 9, depth = 7.

In [12]:
best_model = None
best_result = 0
for est in range(1, 16):
    for depth in range(1, 11):
        for features in ['sqrt', 'log2', None]:
            for bootstrap in [True, False]:
                model_forest = RandomForestClassifier(
                    max_features=features, bootstrap=bootstrap,
                    max_depth=depth, n_estimators=est, random_state=12345
                )
                model_forest.fit(features_train, target_train)
                result = model_forest.score(features_valid, target_valid)
                if result > best_result:
                    best_model_forest = model_forest
                    best_result = result
                    best_depth = depth
                    best_n_estimators = est
                    best_max_features = features
                    best_bootstrap = bootstrap
                           
                            
print(best_result, best_max_features, best_bootstrap, best_n_estimators, best_depth)

0.7993779160186625 None True 10 9


Accuracy лучшей модели равна 0.7993779160186625 с гиперпараметрами max_features = None, bootstrap = True, max_depth = 9, n_estimators = 10

In [13]:
best_model_regression = None
best_result = 0
for solver in ['liblinear', 'newton-cg', 'lbfgs']:
    for iter in range(100, 501):
        model_regression = LogisticRegression(random_state=12345, solver=solver, max_iter=iter)
        model_regression.fit(features_train, target_train)
        result = model_regression.score(features_valid, target_valid)
        if result > best_result:
            best_result = result
            best_model_regression = model_regression
            best_solver = solver
            best_max_iter = iter

print(best_result, best_solver, best_max_iter)

0.7262830482115086 newton-cg 100


Accuracy наилучшей модели равна 0.7262830482115086 с гиперпараметрами solver = newton-cg, max_iter = 100.

<b>Итого

Самый высокий показатель accuracy у модели, обученной алгоритмом случайного леса.

Accuracy =  0.7993779160186625

Гиперпараметры: max_features = None, bootstrap = True, max_depth = 9, n_estimators = 10

## Проверьте модель на тестовой выборке

Проверим эту модель на тестовой выборке.

In [14]:
best_model_forest.score(features_test, target_test)

0.7931570762052877

Accuracy лучшей модели на тестовой выборке равна 0.7931570762052877

## (бонус) Проверьте модели на адекватность

Для проверки используем модель DummyClassifier.

In [15]:
from sklearn.dummy import DummyClassifier

In [17]:
model_check = DummyClassifier(random_state=12345, strategy='most_frequent')
model_check.fit(features_train, target_train)
model_check.score(features_valid, target_valid)

0.6889580093312597

Accuracy модели DummyClassifier ниже, чем у модели RandomForestClassifier (0.6889580093312597 против 0.7931570762052877), значит, модель адекватна.

## Общий вывод

В данных не обнаружили артефактов.

Исследовали модели DecisionTreeClassifier, RandomForestClassifier и LogisticRegression. Наилучшую точность показала модель RandomForestClassifier с гиперпараметрами max_features = None, bootstrap = True, max_depth = 9, n_estimators = 10, ее accuracy - 0.7993779160186625.

Модель RandomForestClassifier с наилучшими гиперапараметрами проверили на тестовой выборке, ее accuracy - 0.7931570762052877.

Модель RandomForestClassifier проверили на адекватность - для проверки использовали модель DummyClassifier, которая показала результат accuracy - 0.6889580093312597, ниже, чем модель RandomForestClassifier.